In [6]:
import torch

import numpy as np
import scipy.io as sio

import pandas as pd
import datetime as dt
import xarray as xr

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


In [7]:
root_dir = '/global/cfs/projectdirs/m1657/liuy351/TallTower/ContrastiveTransformer'


In [8]:
feature_path = '../runs/transformer_embedding.single.pt'

In [9]:
# read features
feature = torch.load(feature_path).numpy()
